多变量Meta分析，不多说手搓！

In [1]:
# 导入必要的包
library(R2jags)

Loading required package: rjags

Loading required package: coda

Linked to JAGS 4.3.2

Loaded modules: basemod,bugs


Attaching package: 'R2jags'


The following object is masked from 'package:coda':

    traceplot




## 研究内相关性已知-随机效应模型

In [49]:
# 录入数据
fulldata <- list(
  n = 5,  # 研究数量
  p = 2,  # 结局数量
  y1 = c(0.47, 0.2, 0.4, 0.26, 0.56),  # PD组均数
  y2 = c(-0.32, -0.6, -0.12, -0.31, -0.39),  # AL组均数
  s1 = c(0.0075, 0.0057, 0.0021, 0.0029, 0.0148),  # PD组方差
  s2 = c(0.0077, 0.0008, 0.0014, 0.0015, 0.0304),  # AL组方差
  s12 = c(0.003, 0.0009, 0.0007, 0.0009, 0.0072)  # PD组和AL组的协方差
)

In [18]:
# 设置汇报参数
params <- c("mean[1]", "mean[2]", "tau2[1]", "tau2[2]")

In [55]:
# 构建模型
model <- function() {
  # 第一层模型构建，真实效应到观察效应
  for (i in 1:n){
    # 先抽取y1
    w1[i] <- 1 / s1[i]
    y1[i] ~ dnorm(m[i, 1], w1[i])  # m[i, 1]是未知参数

    # 再抽取y2，条件分布y2|y1
    rho[i] <- s12[i] / sqrt(s1[i] * s2[i])
    m_y[i] <- m[i, 2] +
              rho[i] * sqrt(s2[i]) / sqrt(s1[i]) * (y1[i] - m[i, 1])  # m[i, 2]是未知参数
    var_y[i] <- s2[i] * (1 - rho[i]^2)
    w2[i] <- 1 / var_y[i]
    y2[i] ~ dnorm(m_y[i], w2[i])
  }

  # 第二层模型构建，真实效应抽样
  for (i in 1:n) {
    # 先抽取m[i, 1]

    m[i, 1] ~ dnorm(mean[1], w[1])

    # 再抽取m[i, 2]
    m_m[i] <- mean[2] + rho_tau * sqrt(tau2[2]) / sqrt(tau2[1]) * (m[i, 1] - mean[1])
    m[i, 2] ~ dnorm(m_m[i], w_m)
  }
  # 方差不需要循环建模
  var_m <- tau2[2] * (1 - rho_tau^2)
  w_m <- 1 / var_m

  # 第三层模型构建，先验层
  # rho_tau先验分布
  a ~ dunif(0, 3.14)
  rho_tau <- cos(a)

  # mean先验分布
  for (j in 1:p) {
    mean[j] ~ dnorm(0, 0.001)
  }

  # tau2先验分布
  for (j in 1:p) {
    w[j] ~ dunif(0, 100)
    tau2[j] <- 1 / w[j]
  }
}

In [56]:
# 设置种子
set.seed(12345)

# 运行模型
result <- jags(
  data = fulldata,
  inits = NULL,
  parameters.to.save = params,
  n.chains = 3,
  n.iter = 50000,
  n.burnin = 20000,
  n.thin = 20,
  model.file = model
)

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 10
   Unobserved stochastic nodes: 15
   Total graph size: 149

Initializing model



In [57]:
print(result)

Inference for Bugs model at "/var/folders/fv/c0dh9y6s7638rh5fdr3lgcmr0000gn/T//RtmpMIING4/model5e153a3b37a.txt", fit using jags,
 3 chains, each with 50000 iterations (first 20000 discarded), n.thin = 20
 n.sims = 4500 iterations saved. Running time = 1.328 secs
         mu.vect sd.vect    2.5%     25%     50%     75%   97.5%  Rhat n.eff
mean[1]    0.359   0.073   0.216   0.313   0.358   0.404   0.504 1.001  4500
mean[2]   -0.341   0.094  -0.533  -0.398  -0.341  -0.283  -0.151 1.001  2600
tau2[1]    0.021   0.016   0.010   0.012   0.015   0.023   0.059 1.001  4500
tau2[2]    0.038   0.037   0.011   0.019   0.028   0.043   0.131 1.002  2200
deviance -27.008   4.416 -33.979 -30.300 -27.557 -24.464 -16.999 1.001  4500

For each parameter, n.eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor (at convergence, Rhat=1).

DIC info (using the rule: pV = var(deviance)/2)
pV = 9.8 and DIC = -17.3
DIC is an estimate of expected predictive error (lower

# 研究内相关性已知-固定效应模型

In [58]:
# 导入必要的包
library(R2jags)

# 录入数据
fulldata <- list(
  n = 5,  # 研究数量
  p = 2,  # 结局数量
  y1 = c(0.47, 0.2, 0.4, 0.26, 0.56),  # PD组均数
  y2 = c(-0.32, -0.6, -0.12, -0.31, -0.39),  # AL组均数
  s1 = c(0.0075, 0.0057, 0.0021, 0.0029, 0.0148),  # PD组方差
  s2 = c(0.0077, 0.0008, 0.0014, 0.0015, 0.0304),  # AL组方差
  s12 = c(0.003, 0.0009, 0.0007, 0.0009, 0.0072)  # PD组和AL组的协方差
)

# 设置汇报参数
params <- c("mean[1]", "mean[2]", "tau2[1]", "tau2[2]")

# 构建模型，固定效应模型
model <- function() {
  # 第一层模型构建，真实效应到观察效应
  for (i in 1:n){
    # 先抽取y1
    w1[i] <- 1 / s1[i]
    y1[i] ~ dnorm(mean[1], w1[i])

    # 再抽取y2，条件分布y2|y1
    rho[i] <- s12[i] / sqrt(s1[i] * s2[i])
    m_y[i] <- mean[2] +
              rho[i] * sqrt(s2[i]) / sqrt(s1[i]) * (y1[i] - mean[1])
    var_y[i] <- s2[i] * (1 - rho[i]^2)
    w2[i] <- 1 / var_y[i]
    y2[i] ~ dnorm(m_y[i], w2[i])
  }

  # 先验层
  for (i in 1:p) {
    mean[i] ~ dnorm(0, 0.001)
  }
}

# 设置种子
set.seed(12345)

# 运行模型
result <- jags(
  data = fulldata,
  inits = NULL,
  parameters.to.save = params,
  n.chains = 3,
  n.iter = 50000,
  n.burnin = 20000,
  n.thin = 20,
  model.file = model
)

print(result)

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 10
   Unobserved stochastic nodes: 2
   Total graph size: 108

Initializing model



Warning message in FUN(X[[i]], ...):
"Failed to set trace monitor for tau2[1]
Variable tau2 not found
"
Warning message in FUN(X[[i]], ...):
"Failed to set trace monitor for tau2[2]
Variable tau2 not found
"


Inference for Bugs model at "/var/folders/fv/c0dh9y6s7638rh5fdr3lgcmr0000gn/T//RtmpMIING4/model5e1522b7bb2f.txt", fit using jags,
 3 chains, each with 50000 iterations (first 20000 discarded), n.thin = 20
 n.sims = 4500 iterations saved. Running time = 0.596 secs
         mu.vect sd.vect   2.5%    25%    50%    75%  97.5%  Rhat n.eff
mean[1]    0.307   0.029  0.252  0.289  0.307  0.326  0.365 1.001  4500
mean[2]   -0.394   0.019 -0.431 -0.406 -0.394 -0.381 -0.358 1.001  4500
deviance  92.880   2.039 90.926 91.442 92.245 93.674 98.488 1.001  4500

For each parameter, n.eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor (at convergence, Rhat=1).

DIC info (using the rule: pV = var(deviance)/2)
pV = 2.1 and DIC = 95.0
DIC is an estimate of expected predictive error (lower deviance is better).


# 研究内相关性未知

In [61]:
# 导入必要的包
library(R2jags)

# 录入数据
fulldata <- list(
  n = 5,  # 研究数量
  p = 2,  # 结局数量
  y1 = c(0.47, 0.2, 0.4, 0.26, 0.56),  # PD组均数
  y2 = c(-0.32, -0.6, -0.12, -0.31, -0.39),  # AL组均数
  s1 = c(0.0075, 0.0057, 0.0021, 0.0029, 0.0148),  # PD组方差
  s2 = c(0.0077, 0.0008, 0.0014, 0.0015, 0.0304)  # AL组方差
)

# 设置汇报参数
params <- c("mean[1]", "mean[2]", "tau2[1]", "tau2[2]")

# 构建模型，相关性未知rho是变量
# 构建模型
model <- function() {
  # 第一层模型构建，真实效应到观察效应
  # 抽取rho
  for (i in 1:n){
    rho[i] ~ dunif(0, 1)
  }

  for (i in 1:n){
    # 先抽取y1
    w1[i] <- 1 / s1[i]
    y1[i] ~ dnorm(m[i, 1], w1[i])  # m[i, 1]是未知参数

    # 再抽取y2，条件分布y2|y1
    m_y[i] <- m[i, 2] +
              rho[i] * sqrt(s2[i]) / sqrt(s1[i]) * (y1[i] - m[i, 1])  # m[i, 2]是未知参数
    var_y[i] <- s2[i] * (1 - rho[i]^2)
    w2[i] <- 1 / var_y[i]
    y2[i] ~ dnorm(m_y[i], w2[i])
  }

  # 第二层模型构建，真实效应抽样
  for (i in 1:n) {
    # 先抽取m[i, 1]

    m[i, 1] ~ dnorm(mean[1], w[1])

    # 再抽取m[i, 2]
    m_m[i] <- mean[2] + rho_tau * sqrt(tau2[2]) / sqrt(tau2[1]) * (m[i, 1] - mean[1])
    m[i, 2] ~ dnorm(m_m[i], w_m)
  }
  # 方差不需要循环建模
  var_m <- tau2[2] * (1 - rho_tau^2)
  w_m <- 1 / var_m

  # 第三层模型构建，先验层
  # rho_tau先验分布
  a ~ dunif(0, 3.14)
  rho_tau <- cos(a)

  # mean先验分布
  for (j in 1:p) {
    mean[j] ~ dnorm(0, 0.001)
  }

  # tau2先验分布
  for (j in 1:p) {
    w[j] ~ dunif(0, 100)
    tau2[j] <- 1 / w[j]
  }
}

# 设置种子
set.seed(12345)

# 运行模型
result <- jags(
  data = fulldata,
  inits = NULL,
  parameters.to.save = params,
  n.chains = 3,
  n.iter = 50000,
  n.burnin = 20000,
  n.thin = 20,
  model.file = model
)

print(result)

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 10
   Unobserved stochastic nodes: 20
   Total graph size: 134

Initializing model

Inference for Bugs model at "/var/folders/fv/c0dh9y6s7638rh5fdr3lgcmr0000gn/T//RtmpMIING4/model5e153a7cf0d0.txt", fit using jags,
 3 chains, each with 50000 iterations (first 20000 discarded), n.thin = 20
 n.sims = 4500 iterations saved. Running time = 1.626 secs
         mu.vect sd.vect    2.5%     25%     50%     75%   97.5%  Rhat n.eff
mean[1]    0.361   0.074   0.222   0.313   0.359   0.407   0.515 1.001  4500
mean[2]   -0.347   0.094  -0.532  -0.405  -0.346  -0.291  -0.163 1.001  4500
tau2[1]    0.021   0.019   0.010   0.012   0.016   0.024   0.063 1.001  4500
tau2[2]    0.039   0.077   0.011   0.019   0.028   0.044   0.121 1.001  4500
deviance -29.216   4.893 -37.413 -32.722 -29.725 -26.148 -18.563 1.001  2400

For each parameter, n.eff is a crude measure of effective sample

# Riley的全相关模型